## Creating Archetypes (New API)

#### Step 1: Setup
First, let's set up our environment and import the necessary libraries:

In [ ]:
from agent_torch.core.llm.archetype import Archetype
from agent_torch.core.llm.mock_llm import MockLLM
import agent_torch.populations.astoria as astoria
import pandas as pd, os

Setup : Covid Cases Data and Unemployment Rate

In [2]:
from utils import get_covid_cases_data

csv_path = "/models/covid/data/county_data.csv"
monthly_cases_kings = get_covid_cases_data(
    csv_path=csv_path, county_name="Kings County"
)

#### Step 2: Initialise LLM Instance

We can use either of the Langchain and Dspy backends to initialise a LLM instance. While these are the frameworks we are supporting currently, you may choose to use your own framework of choice by extending the LLMBackend class provided with AgentTorch.

Let's see how we can use Langchain to initialise an LLM instance

GPT 3.5 Turbo

In [3]:
agent_profile = "You are an helpful agent who is trying to help the user make a decision. Give answer as a single number between 0 and 1, only."
llm_langchain_35 = LangchainLLM(
    openai_api_key=OPENAI_API_KEY, agent_profile=agent_profile, model="gpt-3.5-turbo"
)

#### Step 3: Define an Archetype

In [ ]:
# New API: build Template and Archetype
import agent_torch.core.llm.template as lm

class MyPromptTemplate(lm.Template):
    system_prompt = "You are evaluating willingness based on job profile and context."
    age = lm.Variable(desc="agent age", learnable=True)
    gender = lm.Variable(desc="agent gender", learnable=False)
    soc_code = lm.Variable(desc="job id", learnable=False)
    abilities = lm.Variable(desc="abilities required", learnable=True)
    work_context = lm.Variable(desc="work context", learnable=True)
    def __prompt__(self):
        self.prompt_string = (
            "You are in your {age}'s and are a {gender}. "
            "As a {soc_code}...you have {abilities} and work in {work_context}."
        )
    def __output__(self):
        return "Rate your willingness to continue normal activities, respond in [0, 1] binary decision only."

prompt_template = MyPromptTemplate()
llm = MockLLM()
arch = Archetype(prompt=prompt_template, llm=llm, n_arch=3)

Create an object of the Behavior class

In [ ]:
# Configure external data and preview
base_dir = os.path.dirname(__file__)
jobs_df = pd.read_pickle(os.path.join(base_dir, "../../..", "job_data_clean.pkl"))
arch.configure(external_df=jobs_df, split=2)
arch.sample()

In [7]:
# Define arguments to be used for creating a query for the LLM Instance
kwargs = {
    "month": "January",
    "year": "2020",
    "covid_cases": 1200,
    "device": "cpu",
    "current_memory_dir": "/populations/astoria/conversation_history",
    "unemployment_rate": 0.05,
}

#### Step 4: Compare performance between different Configurations of Archetype

In [ ]:
# Broadcast to population and sample
arch.configure(external_df=jobs_df)
arch.broadcast(population=astoria, match_on="soc_code")
arch.sample(print_examples=1)

# Optional: Optimize with P3O
from agent_torch.optim import P3O
opt = P3O(arch.parameters(), archetype=arch)
for _ in range(2):
    arch.sample(print_examples=1)
    opt.step()
    opt.zero_grad()